# Kubernetes

# Kubernetes Version (Current)

In [1]:
from kubernetesTool import executeKubectlCmd
from kubernetesTool import toCmd
from kubernetesTool import executeLocalCmd
from kubernetesTool import createConfig
from kubernetesTool import getPort
from kubernetesTool import executePeerCmd

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

In [1]:
!pip install ipynb
from ipynb.fs.full.facturxWidget import *

Text(value='15', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Dropdown(description='FILTER', index=1, layout=Layout(width='500px'), options=(False, True), style=Description…

Dropdown(description='FILTER', index=3, layout=Layout(width='500px'), options=('/supplyChainTradeTransaction',…

Output()

HTML(value='', description=' ', layout=Layout(width='500px'), style=DescriptionStyle(description_width='250px'…

...

Output()

Text(value='15', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Output()

Text(value='14', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Output()

## check status peers

In [727]:
from kubernetesTool import executeKubectlCmd
_res = executeKubectlCmd(['get', 'pods', '-o', 'json'], decode='json')

for item in _res['items']:
    labels=item['metadata']['labels']
    #status=item['status']['initContainerStatuses'][0]
    #print(labels,status['state'],status['restartCount'],status['name'],status['ready'])
    print(labels,item['status']['phase'])
    #print(labels)

{'app': 'aabby-0', 'pod-template-hash': '6859c54c76'} Running
{'app': 'aabby-1', 'pod-template-hash': '5ff5b8677f'} Running
{'app': 'babby-0', 'pod-template-hash': '7d7bcd4f45'} Running
{'app': 'babby-1', 'pod-template-hash': '56f9fc8dcd'} Running
{'app': 'babby-2', 'pod-template-hash': '97b65984c'} Running
{'app': 'babby-3', 'pod-template-hash': '67c5fc7f6'} Running
{'app': 'cc-deployer', 'pod-template-hash': '55c958b4d4'} Running
{'app': 'hl-fabric-data-share', 'pod-template-hash': '6849487f76'} Running
{'app': 'invoice-chaincode-external', 'pod-template-hash': '756d778fb'} Running
{'app': 'invoice-net-chaincode-external', 'pod-template-hash': '77fc764779'} Running
{'app': 'orderer-org1-scray-org', 'pod-template-hash': '5cb87587c4'} Running
{'app': 'peer80', 'pod-template-hash': '8774cd6cb'} Running
{'app': 'peer81', 'pod-template-hash': '66554b66f'} Running
{'app': 'peer90', 'pod-template-hash': '6bd5f7bd99'} Running
{'app': 'peer91', 'pod-template-hash': '5976d6d958'} Running
{'app

In [103]:
kubernetespeer('invoice-chaincode-external').__dict__

Error from server (NotFound): services "invoice-chaincode-external" not found
Error from server (NotFound): services "invoice-chaincode-external" not found


{'name': 'invoice-chaincode-external',
 'pod_name': 'invoice-chaincode-external-847cd44954-w2ddx',
 'listen_port': 'Command \'[\'/home/jovyan/work/usr/bin/kubectl\', \'get\', \'service\', \'invoice-chaincode-external\', \'-o\', \'jsonpath="{.spec.ports[?(@.name==\\\'peer-listen\\\')].nodePort}"\']\' returned non-zero exit status 1.',
 'gossip_port': 'Command \'[\'/home/jovyan/work/usr/bin/kubectl\', \'get\', \'service\', \'invoice-chaincode-external\', \'-o\', \'jsonpath="{.spec.ports[?(@.name==\\\'peer-gossip\\\')].nodePort}"\']\' returned non-zero exit status 1.'}

In [ ]:
_res['items'][3]['metadata']['labels']
_res['items'][0]['status']['phase']

In [104]:
#30489
kubernetesTool.getPort(peername='invoice-chaincode-external', name = prefix + '-listen')

NameError: name 'prefix' is not defined

In [ ]:
kubernetespeer(name='invoice-net-chaincode-external').__dict__

In [108]:
#_res['items'][5]['status']['containerStatuses']
_res['items'][5]

{'apiVersion': 'v1',
 'kind': 'Pod',
 'metadata': {'creationTimestamp': '2021-10-22T09:19:35Z',
  'generateName': 'invoice-net-chaincode-external-576f8cbdd-',
  'labels': {'app': 'invoice-net-chaincode-external',
   'pod-template-hash': '576f8cbdd'},
  'managedFields': [{'apiVersion': 'v1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:generateName': {},
      'f:labels': {'.': {}, 'f:app': {}, 'f:pod-template-hash': {}},
      'f:ownerReferences': {'.': {},
       'k:{"uid":"6eb1c3d5-284f-4d41-9619-4ee45df70947"}': {'.': {},
        'f:apiVersion': {},
        'f:blockOwnerDeletion': {},
        'f:controller': {},
        'f:kind': {},
        'f:name': {},
        'f:uid': {}}}},
     'f:spec': {'f:containers': {'k:{"name":"invoice-net-chaincode-external"}': {'.': {},
        'f:env': {'.': {},
         'k:{"name":"CHAINCODE_ID"}': {'.': {}, 'f:name': {}, 'f:value': {}},
         'k:{"name":"CHAINCODE_SERVER_ADDRESS"}': {'.': {},
          'f:name': {},
          

In [78]:
for item in _res['items']:
    labels=item['metadata']['labels']
    if labels['app'] == 'invoice-net-chaincode-external':
        print(item)

{'apiVersion': 'v1', 'kind': 'Pod', 'metadata': {'creationTimestamp': '2021-10-22T09:19:35Z', 'generateName': 'invoice-net-chaincode-external-576f8cbdd-', 'labels': {'app': 'invoice-net-chaincode-external', 'pod-template-hash': '576f8cbdd'}, 'managedFields': [{'apiVersion': 'v1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:labels': {'.': {}, 'f:app': {}, 'f:pod-template-hash': {}}, 'f:ownerReferences': {'.': {}, 'k:{"uid":"6eb1c3d5-284f-4d41-9619-4ee45df70947"}': {'.': {}, 'f:apiVersion': {}, 'f:blockOwnerDeletion': {}, 'f:controller': {}, 'f:kind': {}, 'f:name': {}, 'f:uid': {}}}}, 'f:spec': {'f:containers': {'k:{"name":"invoice-net-chaincode-external"}': {'.': {}, 'f:env': {'.': {}, 'k:{"name":"CHAINCODE_ID"}': {'.': {}, 'f:name': {}, 'f:value': {}}, 'k:{"name":"CHAINCODE_SERVER_ADDRESS"}': {'.': {}, 'f:name': {}, 'f:value': {}}, 'k:{"name":"FABRIC_LOGGING_SPEC"}': {'.': {}, 'f:name': {}, 'f:value': {}}}, 'f:image': {}, 'f:imagePullPolicy': {}, 'f:n

## addPeer

In [35]:
def addPeer(peername):  
    if peername in peers.keys():
        return peers[peername]
    # Create configuration for new peer
    executeLocalCmd(['/bin/bash','-c','cd /home/jovyan/scray-ledger/containers/ && /home/jovyan/scray-ledger/containers/configure-deployment.sh -n ' + peername])
    # Start service
    executeKubectlCmd(['apply', '-f', '/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer-service.yaml'])
    # Create peer configuration
    res = createConfig(peername=peername,
                       peer_listen_port=getPort(peername=peername, name='peer-listen'),
                       peer_gossip_port=getPort(peername=peername, name='peer-gossip'))
    # Start new peer
    executeKubectlCmd(['apply', '-f','/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer.yaml'])
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer
    
# create, delete numbered peers
def createPeers(base_name,start=0, end=10):
    for x in range(start,end):
        peername = base_name + '-' + str(x)
        addPeer(peername)
         
def deletePeers(name):
    for x in range(0,10):
        peerneme = name + str(x)
        deletePeer(peerneme)    
  
def deleteAllPeers(config):
    for item in config['items']:
        try:
            if 'org_name' in item['data'].keys():
                deletePeer(item['data']['org_name'])
        except Exception as e:
            print(item['data'])
            

### config            
            
import subprocess
def deleteConfig(peername=''):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + peername])
        return str(output)
    except Exception as e:
        return str(e)  
        
    
def createConfig(peername='',peer_listen_port='',peer_gossip_port=''):
    try:  
        peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                          '--from-literal=hostname=' + peer_host_name,
                                          '--from-literal=org_name=' + peername,
                                          '--from-literal=data_share=hl-fabric-data-share-service:80',
                                          '--from-literal=CORE_PEER_ADDRESS=peer0.' + peer_host_name + ':' + peer_listen_port,
                                          '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=' + peer_host_name + ':' + peer_gossip_port,
                                          '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
        return str(output)
    except Exception as e:
        return str(e)      
    
  
##### execute

def executePeerCmd(pod_name,cmd, cli='scray-peer-cli'):
    try:  
        #cmd1 = "source /root/.bashrc && "
        cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd]
        #cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c'] + cmd
        print(toCmd(cmd1))
        output  = subprocess.check_output(cmd1)
        #output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec', '--stdin' ,'--tty', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd])
        return output.decode('ascii')
    except Exception as e:
        return str(e)

In [20]:
import kubernetesTool

class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name
        self.pod_name = kubernetesTool.getPod(name)
        self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        self.gossip_port = kubernetesTool.getPort(peername=name, name = prefix + '-gossip')
        # ????????
        #print('kubernetespeer addBashEnvToPeer')
        #addBashEnvToPeer(self)
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)      
        
class ordererpeer(kubernetespeer):
    def __init__(self, name='',prefix='peer'):
        super().__init__(name,prefix)
        self.host = 'orderer.example.com'
        self.orderer_intern = 7050
        
    # can change
    def get_ip(self, app='orderer-org1-scray-org'):
        return executeKubectlCmd(['get', 'pods','-l', 'app=' + app,'-o','jsonpath=\'{.items[*].status.podIP}\''])   
    

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

In [19]:
kubernetespeer('buyer')

NameError: name 'kubernetesTool' is not defined

In [6]:
def toCmd(strlist):
    return ''.join((str(e) + ' ') for e in strlist)[:-1]

#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME         
def joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip='10.15.136.41'):          
    cmd = toCmd(['/mnt/conf/peer_join.sh', orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip])
    print(cmd)
    return executePeerCmd(peer.pod_name, cmd)

def endorse_peer(peer=None, orderer_ip='',channel_name='',shared_fs_host='',ext_peer_ip='10.15.136.41'):
    print(peer.name)
    peername = peer.name
    peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
    cmd = toCmd(['/mnt/conf/orderer/scripts/inform_existing_nodes.sh',orderer_ip,channel_name,peer.name,shared_fs_host,ext_peer_ip,peer_host_name]) 
    #print(cmd)
    return executePeerCmd(orderer.pod_name,cmd,cli='scray-orderer-cli')

def add_channel(name):
    cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050']
    cmd = toCmd(cmd1)
    return executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

## create peers

In [479]:
orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer')


orderer_ip=orderer.get_ip()
orderer_hostname=orderer.host
#orderer_port=orderer.orderer_intern
orderer_port= orderer.listen_port

channel_name='super55'
shared_fs_host='10.14.128.38:30080'

In [10]:
def endorse_and_join(peer):    
    res1 = endorse_peer(peer=peer,orderer_ip=orderer_ip,channel_name=channel_name,shared_fs_host='10.14.128.38:30080')   
    res2  = joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host)
    return res1,res2

In [480]:
peers = {}

In [698]:
#createPeers('babby',start=0, end=2)
createPeers('babby',start=2, end=4)

In [699]:
## endorse and join

import ipywidgets as widgets
out2 = widgets.Output()

with out2:
    for key, value in peers.items():
        endorse_and_join(value)

In [700]:
out2

Output(outputs=({'name': 'stdout', 'text': "babby-0\n/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-sc…

In [16]:
peername='zalya-3'
peer = kubernetespeer(name=peername)
peers[peername] = peer

In [ ]:
'zalya-0'

In [ ]:
peers['zalya-1'].__dict__
#peers

In [ ]:
addPeer('zalya-3')

In [ ]:
orderer.__dict__

In [ ]:
kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/inform_existing_nodes.sh $ORDERER_IP $CHANNEL_NAME $PEER_NAME $SHARED_FS_HOST $EXT_PEER_IP $PEER_HOST_NAME

In [ ]:
EER_POD_NAME=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
 ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
 ORDERER_PORT=7050
 PEER_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
 kubectl exec --stdin --tty $PEER_POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $ORDERER_PORT $CHANNEL_NAME $SHARED_FS_HOST $EXT_PEER_IP

In [ ]:
endorse_and_join(peers['zalya-3'])

In [ ]:
installAndAproveChaincode(peers['zalya-1'],orderer,channel_name)


In [ ]:
#out2.outputs[1]['text'].splitlines()

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-5f97c57d44-j5mbs -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super40 kalya-0 10.14.128.38:30080 10.15.136.41 kalya-0.kubernetes.research.dev.seeburger.de

In [ ]:
name='peer-gossip'
callopt= 'jsonpath=\"{.spec.ports[?(@.name==\'' + name + '\')].nodePort}\"'
callopt

## install chaincode

In [133]:
def installAndAproveChaincode(peer,orderer,channel_name,cc_host='10.14.128.38',pkgid='',cc_hostname='asset-transfer-basic.org1.example.com',cc_label='basic_1.0',shared_fs='kubernetes.research.dev.seeburger.de:30080', sequence=1):
    #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK
    #strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.orderer_intern), channel_name ]
    strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.listen_port), channel_name,pkgid,cc_hostname, cc_label,shared_fs, str(sequence) ]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

def commitChaincode(peer,channel_name,sequence=1,pkgid=''):
    strlist = ['/mnt/conf/peer/cc_commit.sh', channel_name, str(sequence),pkgid ]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

In [701]:
for key, peer in peers.items():
    executePeerCmd(peer.pod_name,'apk add curl')
    strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh', '-o', '/mnt/conf/install_and_approve_cc.sh']
    cmd = toCmd(strlist)
    print(cmd)
    output = executePeerCmd(peer.pod_name,  cmd)
    executePeerCmd(peer.pod_name,'chmod 755 /mnt/conf/install_and_approve_cc.sh')
    strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh', '-o', '/mnt/conf/peer/cc_commit.sh']
    cmd = toCmd(strlist)
    print(cmd)
    output = executePeerCmd(peer.pod_name,  cmd)
    executePeerCmd(peer.pod_name,'chmod 755 /mnt/conf/peer/cc_commit.sh')

/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c apk add curl
curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh -o /mnt/conf/install_and_approve_cc.sh
/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh -o /mnt/conf/install_and_approve_cc.sh


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1557  100  1557    0     0   6129      0 --:--:-- --:--:-- --:--:--  6129


/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c chmod 755 /mnt/conf/install_and_approve_cc.sh
curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh -o /mnt/conf/peer/cc_commit.sh
/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh -o /mnt/conf/peer/cc_commit.sh


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   523  100   523    0     0   2410      0 --:--:-- --:--:-- --:--:--  2410


/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c chmod 755 /mnt/conf/peer/cc_commit.sh
/home/jovyan/work/usr/bin/kubectl exec -t babby-1-56f9fc8dcd-r7ggz -c scray-peer-cli -- /bin/sh -c apk add curl
curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh -o /mnt/conf/install_and_approve_cc.sh
/home/jovyan/work/usr/bin/kubectl exec -t babby-1-56f9fc8dcd-r7ggz -c scray-peer-cli -- /bin/sh -c curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh -o /mnt/conf/install_and_approve_cc.sh


...

/home/jovyan/work/usr/bin/kubectl exec -t babby-2-97b65984c-bvldw -c scray-peer-cli -- /bin/sh -c chmod 755 /mnt/conf/peer/cc_commit.sh
/home/jovyan/work/usr/bin/kubectl exec -t babby-3-67c5fc7f6-zlmgw -c scray-peer-cli -- /bin/sh -c apk add curl
curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh -o /mnt/conf/install_and_approve_cc.sh
/home/jovyan/work/usr/bin/kubectl exec -t babby-3-67c5fc7f6-zlmgw -c scray-peer-cli -- /bin/sh -c curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh -o /mnt/conf/install_and_approve_cc.sh


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1557  100  1557    0     0  47181      0 --:--:-- --:--:-- --:--:-- 47181


/home/jovyan/work/usr/bin/kubectl exec -t babby-3-67c5fc7f6-zlmgw -c scray-peer-cli -- /bin/sh -c chmod 755 /mnt/conf/install_and_approve_cc.sh
curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh -o /mnt/conf/peer/cc_commit.sh
/home/jovyan/work/usr/bin/kubectl exec -t babby-3-67c5fc7f6-zlmgw -c scray-peer-cli -- /bin/sh -c curl https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh -o /mnt/conf/peer/cc_commit.sh


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   523  100   523    0     0  16343      0 --:--:-- --:--:-- --:--:-- 16343


/home/jovyan/work/usr/bin/kubectl exec -t babby-3-67c5fc7f6-zlmgw -c scray-peer-cli -- /bin/sh -c chmod 755 /mnt/conf/peer/cc_commit.sh


In [492]:
 peernames

['babby-0', 'babby-1']

In [490]:
shared_fs='kubernetes.research.dev.seeburger.de:30080'
cc_hostname='invoice-net.org1.example.com'
cc_host='10.14.128.38'
cc_service_name='hl-fabric-cc-external-invoice-net'
cc_port = executeKubectlCmd(['get', 'service', cc_service_name, '-o', 'jsonpath="{.spec.ports[?(@.name==\'chaincode\')].nodePort}"'], decode='json')
cc_label='basic_1.0'

In [139]:
cc_port

'30489'

In [491]:
strlist = ['curl','-s','--user','scray:scray','http://' + shared_fs + '/cc_descriptions/' + cc_hostname + '_' + cc_label + '/description-hash.json']
cmd = toCmd(strlist)
output = executePeerCmd(peer.pod_name,  cmd)
pkgid = json.loads(output[:-1])['description-hash']

/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c curl -s --user scray:scray http://kubernetes.research.dev.seeburger.de:30080/cc_descriptions/invoice-net.org1.example.com_basic_1.0/description-hash.json


In [702]:
for key, peer in peers.items():
    installAndAproveChaincode(peer,orderer,channel_name,cc_host=cc_host,pkgid=pkgid,cc_hostname=cc_hostname,cc_label=cc_label,shared_fs=shared_fs, sequence=1)

/mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1
/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   431  100   431    0     0  15962      0 --:--:-- --:--:-- --:--:-- 15962
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2021-11-23 14:40:41.526 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [cc68aa153e33599325b9fe7f906cc58a999331dc02195868412d81d85d9ecec5] committed with status (VALID) at peer0.babby-0.kubernetes.research.dev.seeburger.de:30576


/mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1
/home/jovyan/work/usr/bin/kubectl exec -t babby-1-56f9fc8dcd-r7ggz -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   431  100   431    0     0  30785      0 --:--:-- --:--:-- --:--:-- 30785
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2021-11-23 14:40:44.990 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [4f8218d8611ed4d844832e2744e0459db4dd2c75a7faebe29a5e15f2931944a7] committed with status (VALID) at peer0.babby-1.kubernetes.research.dev.seeburger.de:31216


/mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1
/home/jovyan/work/usr/bin/kubectl exec -t babby-2-97b65984c-bvldw -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   431  100   431    0     0  13903      0 --:--:-- --:--:-- --:--:-- 13903
2021-11-23 14:40:45.927 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77\022\tbasic_1.0" > 
2021-11-23 14:40:45.927 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chaincode code package identifier: basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77
2021-11-23 14:40:48.362 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [01a1f784f2949cf6e214ce2ca258f8cb3f7d95c907b106f8ee278adf9018ebfc] committed with status (VALID) at peer0.babby-2.kubernetes.research.dev.seeburger.de:31025


/mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1
/home/jovyan/work/usr/bin/kubectl exec -t babby-3-67c5fc7f6-zlmgw -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super55 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77 invoice-net.org1.example.com basic_1.0 kubernetes.research.dev.seeburger.de:30080 1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   431  100   431    0     0  53875      0 --:--:-- --:--:-- --:--:-- 53875
2021-11-23 14:40:49.300 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77\022\tbasic_1.0" > 
2021-11-23 14:40:49.300 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chaincode code package identifier: basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77
2021-11-23 14:40:51.795 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [3d51e862c8d0e371cd94222541298f98a315cfda6894269e2f67d732a0b6e76c] committed with status (VALID) at peer0.babby-3.kubernetes.research.dev.seeburger.de:30789


In [703]:
peernames = list(peers.keys())
peernames
peer = peers[peernames[0]]
#peer = list(peers.values())[0]
commitChaincode(peer,channel_name,sequence=1,pkgid=pkgid)

/mnt/conf/peer/cc_commit.sh super55 1 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77
/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc_commit.sh super55 1 basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77


2021-11-23 14:43:38.902 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [c6f26b83cc2ed85a696d270a8b85764d6d260dd5a8a9836e821557ba28eb22fa] committed with status (VALID) at peer0.babby-0.kubernetes.research.dev.seeburger.de:30576


'Installed chaincode\nInstalled chaincodes on peer:\nPackage ID: basic_1.0:a16a9f7ab92fae0f5268f0f289e7158de179a1a2d3dfbcf3cf480fe41886da77, Label: basic_1.0\n'

In [ ]:
#channel_name
#orderer.__dict__
#peers['ralya-0'].pod_name
peers['zalya-0'].__dict__ , channel_name

In [ ]:
installAndAproveChaincode(peers['zalya-1'],orderer,channel_name,sequence=2)


In [ ]:
#commitChaincode(peers['zalya-1'],channel_name,sequence=2)
chaincode_init(peers['zalya-1'],channel_name)

In [ ]:
peers['zalya-2'].pod_name

In [ ]:
def downloadToPeer(peer, orgs, execute_org='schalke3'):
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name=getCrtName(org))))   

In [704]:
peers

{'babby-0': <__main__.kubernetespeer at 0x7fe3ccafa910>,
 'babby-1': <__main__.kubernetespeer at 0x7fe3ccafa7c0>,
 'babby-2': <__main__.kubernetespeer at 0x7fe3cd78e9d0>,
 'babby-3': <__main__.kubernetespeer at 0x7fe3cc9bd9d0>}

In [18]:
peer = peers['zalya-0']
executePeerCmd(peer.pod_name,'apk add curl')

/home/jovyan/work/usr/bin/kubectl exec -t zalya-0-6cf64f96b4-vv4mp -c scray-peer-cli -- /bin/sh -c apk add curl


'fetch http://dl-cdn.alpinelinux.org/alpine/v3.12/main/x86_64/APKINDEX.tar.gz\nfetch http://dl-cdn.alpinelinux.org/alpine/v3.12/community/x86_64/APKINDEX.tar.gz\n(1/1) Installing curl (7.79.1-r0)\nExecuting busybox-1.31.1-r16.trigger\nOK: 29 MiB in 28 packages\n'

In [20]:
peer.pod_name

'zalya-0-6cf64f96b4-vv4mp'

In [ ]:
peer=kubernetespeer(name='oalya-0')
channel_name='super45'

In [ ]:
peer.name, channel_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/cc-basic-interaction.sh  super43

In [70]:
#Call init method in chaincode
def chaincode_init(peer,channel_name):
    strlist = ['/mnt/conf/peer/cc-basic-interaction.sh', channel_name ]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

chaincode_init(peer,channel_name)

/mnt/conf/peer/cc-basic-interaction.sh super55
/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc-basic-interaction.sh super55


Error: endorsement failure during invoke. response: status:500 message:"make sure the chaincode basic has been successfully defined on channel super55 and try again: chaincode definition for 'basic' exists, but chaincode is not installed" 
Error: endorsement failure during query. response: status:500 message:"make sure the chaincode basic has been successfully defined on channel super55 and try again: chaincode definition for 'basic' exists, but chaincode is not installed" 
command terminated with exit code 1


"Command '['/home/jovyan/work/usr/bin/kubectl', 'exec', '-t', 'aabby-0-6859c54c76-gvdqw', '-c', 'scray-peer-cli', '--', '/bin/sh', '-c', '/mnt/conf/peer/cc-basic-interaction.sh super55']' returned non-zero exit status 1."

In [ ]:
externalbuilder/builds/basic_1.0-21ee1256844f847e196af6aec17c5cafa76d8bd652d6ef42c3058e52571030b4/release/chaincode/server/c

In [ ]:
peers['nalya-0'].pod_name

In [ ]:
CHANNEL_NAME='super44'
#ORDERER_NAME=orderer.example.com
IP_CC_SERVICE='10.14.128.38'         # Host where the chaincode is running
#PEER_POD=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
#ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')
#ORDERER_LISTEN_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
#ORDERER_HOST=orderer.example.com
EXT_PEER_IP="10.14.128.38"
PEER_POD="nalya-0-5494459f47-v7msj"

ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
ORDERER_PORT=30081
ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')

kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $ORDERER_IP $ORDERER_HOST $ORDERER_PORT $CHANNEL_NAME 

In [ ]:
#peers.keys()
#[elem for elem in peers.values()]
list(peers.values())[0].__dict__

# invoice usecase

In [497]:
peernames = list(peers.keys())
peernames

['babby-0', 'babby-1']

## invoice functions

In [417]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh  $CHANNEL_NAME $INVOICE_ID

def get_certificate_string(peername,user='User1'):
    return 'x509::CN=' + user + '@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

def chaincode_add_invoice(peer,channel_name,invoice_id,buyer):
    strlist = ['/mnt/conf/peer/add-invoice.sh', channel_name,invoice_id,get_certificate_string(buyer)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 


In [2]:
import json

def getHostname(peer):
    return executePeerCmd(peer.pod_name,'echo $HOSTNAME')[:-1]

def getExport_MSPCONFIGPATH(hostname,user='User1'):
    #user = 'User1'
    #user = 'hans'
    #path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/User1@' + hostname + '/msp/'
    path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + user + '@' + hostname + '/msp/'
    return 'export CORE_PEER_MSPCONFIGPATH=' + path
    
def getArgList(asset):
    if isinstance(asset, dict):
        asset = asset.values()
    return '[' + ''.join(('\"' + str(e) + '\",') for e in asset)[:-1] + ']' 

def invoke(peer,callopt,user='User1'):
    global _out
    _out = widgets.Output()
    with _out:
        try:
            export_cmd = getExport_MSPCONFIGPATH(getHostname(peer),user=user)
            cmd0 = str('\'' + callopt + "\'")
            strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
            cmd = toCmd(strlist)
            cmd = export_cmd + '  && ' + cmd
            print(cmd)  
            output = executePeerCmd(peer.pod_name, cmd)
            return output,_out
        except Exception as e:
            print(e)
            return output,_out
        
def query(peer, callopt, user='User1'):
    global _out
    _out = widgets.Output()
    with _out:
        try:
            export_cmd = getExport_MSPCONFIGPATH(getHostname(peer),user=user)
            #hostname = getHostname(peer)
            #path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + 'User1' + '@' + hostname + '/msp/'
            cmd0 = str('\'' + callopt + "\'")
            strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0]
            #strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0,'--cafile',path + 'cacerts/ca.zalya-0.kubernetes.research.dev.seeburger.de-cert.pem' , '--certfile',path + 'signcerts/User1@zalya-0.kubernetes.research.dev.seeburger.de-cert.pem' ,'--keyfile', path + 'keystore/priv_sk']
            cmd = toCmd(strlist)
            cmd = export_cmd + '  && ' + cmd
            print(cmd)
            output = executePeerCmd(peer.pod_name, cmd)
            #output = executePeerCmd(peer.pod_name, cmd)           
            return json.loads(output),_out
        except Exception as e:
            print(e)
            return output,_out
    
def getAllKeys(peer):
    callopt= '{"function":"GetAllKeys","Args":[' + ']}'
    return query(peer,callopt)       

def getRoleTransactions(peer):
    callopt= '{"function":"GetRoleTransactions","Args":[' + ']}'
    return query(peer,callopt)   
    
def getEmptyInvoice(peer):
    callopt= '{"function":"GetEmptyInvoice","Args":[' + ']}'
    return query(peer,callopt)       
    
def createInvoice(peer,arglist='', user='User1'):
    callopt= '{"function":"CreateInvoice","Args":' + arglist + '}'
    return invoke(peer,callopt,user=user)    
    
def listInvoices(peer, user='User1'):
    callopt= '{"function":"ListInvoices","Args":[' + ']}'
    return query(peer,callopt,user=user)    
    
def listInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ListInvoice","Args":' + getArgList([id]) + '}'
    return query(peer,callopt,user=user)    
    
def receivedInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedInvoice","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)     
    
def receivedOrder(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedOrder","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)   

def receivedPayment(peer, id='asset1', payer='', user='User1'):
    callopt= '{"function":"ReceivedPayment","Args":' + getArgList([id,payer]) + '}'
    return invoke(peer,callopt,user=user)   

def taxReceived(peer, id='asset1', user='User1'):
    callopt= '{"function":"TaxReceived","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)    
    
    
def assetExists(peer, id='asset1'):
    #callopt= '{"function":"AssetExists","Args":[' + '\"' + id + '\"' + ']}'
    callopt= '{"function":"AssetExists","Args":' + getArgList([id]) + '}'
    return query(peer,callopt)
        
def deleteAsset(peer, asset='asset1'):
    callopt= '{"function":"DeleteAsset","Args":[' + '\"' + asset + '\"' + ']}'
    return invoke(peer,callopt)
    


def readAssetTest(peer, asset='asset1'):
    callopt= '{"function":"ReadAssetTest","Args":[' + '\"' + asset + '\"' + ']}'
    return query(peer,callopt)
    
def updateAsset(peer,arglist=None):
    callopt= '{"function":"UpdateAsset","Args":' + getArgList(asset) + '}'
    return invoke(peer,callopt)


def getSubmittingClientIdentity(peer, user='User1'):
    callopt= '{"function":"GetSubmittingClientIdentity","Args":[' + ']}'
    return query(peer,callopt,user=user)

def getMSPID(peer):
    callopt= '{"function":"GetMSPID","Args":[' + ']}'
    return query(peer,callopt)   


def appendRole(peer,user='',role='Buyer'):
    #print(getArgList([role,name]))
    #callopt= '{"function":"AppendRole","Args":' + getArgList([role,name]) + '}'
    name = getSubmittingClientIdentity(peer, user=user)[:-1]
    callopt= '{"function":"AppendRole","Args":' + getArgList([name,role]) + '}'
    return invoke(peer,callopt)

def getRoles(peer,name=''):
    #callopt= '{"function":"GetRoles","Args":[' + ']}'
    callopt= '{"function":"GetRoles","Args":' + getArgList([name]) + '}'
    return query(peer,callopt)

def getAllRoles(peer):
    callopt= '{"function":"GetAllRoles","Args":[' + ']}'
    return query(peer,callopt)




### Enroll users

In [33]:
import subprocess
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
   

def uploadFromLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                          '-T', local_path + '/' + local_name, 
                                          'http://' + host + '/' + remote_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)   
                    
#downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

def downloadToLocalCmdList(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', remote_name='' ,remote_path='/'):  
    return ['curl', '--user',user + ':' + pwd, 'http://' + host + '/' + remote_path + '/' + remote_name, '-o', local_path + '/' + remote_name]


In [631]:
# download walletcreator to peers
def uploadWalletToolToSharepoint():
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator/target', local_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar' ,remote_path='/')
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator', local_name='cert-creator.sh' ,remote_path='upload')

def installPeerWalletTool(peer):
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar'))) 
    #executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='cert-creator.sh'))) 
    executePeerCmd(peer.pod_name,'apk add openssl')
    executePeerCmd(peer.pod_name,'apk add openjdk8-jre')
    # chmod 755
    
def createPeerUser(peer,user, admin='admin',
    cert='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/ca.kubernetes.research.dev.seeburger.de-cert.pem',
    key='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/priv_sk'):
        cmd = ['/tmp/cert-creator.sh','--cacert',cert,'--cakey',key,'--new-user-crt',user,'--organizational-unit',admin,
               '--create-wallet','true','--wallet-creator-lib-path', '/tmp']
        return executePeerCmd(peer.pod_name,toCmd(cmd))
    
def installWalletTool():
    _out = widgets.Output()
    with _out:
        for key, peer in peers.items():
            installPeerWalletTool(peer)
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/cert-creator.sh', '-o', '/tmp/cert-creator.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(peer.pod_name,  cmd)
            executePeerCmd(peer.pod_name,'chmod 755 /tmp/cert-creator.sh')
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/create-user.sh', '-o', '/tmp/create-user.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(peer.pod_name,  cmd)
            executePeerCmd(peer.pod_name,'chmod 755 /tmp/create-user.sh')
    
def addUser(peer,user):
    _out = widgets.Output()
    with _out:
        strlist = ['/tmp/create-user.sh', user, peer.name + '.kubernetes.research.dev.seeburger.de']
        cmd = toCmd(strlist)
        print(cmd)
        output = executePeerCmd(peer.pod_name, 'cd /tmp && ' + cmd)    

In [705]:
installWalletTool()

In [677]:
peer=peers[peernames[0]]
addUser(peers[peernames[0]],'Tony.Dubois')    
addUser(peers[peernames[0]],'Stephanie.Hoarau')
addUser(peers[peernames[0]],'Alexandre.Payet')
appendRole(peer,user='Tony.Dubois',role='Seller')
appendRole(peer,user='Stephanie.Hoarau',role='Buyer')
appendRole(peer,user='Alexandre.Payet',role='Buyer')

peer=peers[peernames[1]]
addUser(peers[peernames[1]],'Emma.Steiner')    
appendRole(peer,user='Emma.Steiner',role='Factor')

peer=peers[peernames[2]]
addUser(peers[peernames[2]],'Pedro.Sanchez')  
appendRole(peer,user='Pedro.Sanchez',role='Buyer')
appendRole(peer,user='Pedro.Sanchez',role='Seller')

peer=peers[peernames[3]]
addUser(peers[peernames[3]],'Auguste.Boucher')   
appendRole(peer,user='Auguste.Boucher',role='TaxInspector')

In [726]:
getAllRoles(peer)

([{'name': '(x509::CN=Alexandre.Payet@babby-0.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-0.kubernetes.research.dev.seeburger.de,O=babby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DEn,)',
   'roles': ['Buyer']},
  {'name': '(x509::CN=Auguste.Boucher@babby-3.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-3.kubernetes.research.dev.seeburger.de,O=babby-3.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DEn,)',
   'roles': ['TaxInspector']},
  {'name': '(x509::CN=Emma.Steiner@babby-1.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-1.kubernetes.research.dev.seeburger.de,O=babby-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DEn,)',
   'roles': ['Factor']},
  {'name': '(x509::CN=Pedro.Sanchez@babby-2.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-2.ku

In [683]:
getSubmittingClientIdentity(peers[peernames[0]], user='Tony.Dubois')[0][:-1].split(',')

['x509::CN=Tony.Dubois@babby-0.kubernetes.research.dev.seeburger.de',
 'OU=admin',
 'O=kubernetes.research.dev.seeburger.de::CN=ca.babby-0.kubernetes.research.dev.seeburger.de',
 'O=babby-0.kubernetes.research.dev.seeburger.de',
 'L=Bretten',
 'ST=Baden',
 'C=DE']

### Monitoring

In [676]:
peers

{'babby-0': <__main__.kubernetespeer at 0x7fe3ccafa910>,
 'babby-1': <__main__.kubernetespeer at 0x7fe3ccafa7c0>}

In [624]:
getRoleTransactions(peer)[0]

[{'role': 'Buyer',
  'transactions': ['main.(*SmartContract).ListInvoice',
   'main.(*SmartContract).ListInvoices',
   'main.(*SmartContract).ReceivedInvoice',
   'main.(*SmartContract).ReceivedOrder']},
 {'role': 'Seller',
  'transactions': ['main.(*SmartContract).ListInvoice',
   'main.(*SmartContract).ListInvoices',
   'main.(*SmartContract).CreateInvoice',
   'main.(*SmartContract).TransferInvoice',
   'main.(*SmartContract).ReceivedPayment']},
 {'role': 'Factor',
  'transactions': ['main.(*SmartContract).ListInvoice',
   'main.(*SmartContract).ListInvoices',
   'main.(*SmartContract).TransferInvoice',
   'main.(*SmartContract).ReceivedPayment']},
 {'role': 'TaxInspector',
  'transactions': ['main.(*SmartContract).ListInvoice',
   'main.(*SmartContract).ListInvoices',
   'main.(*SmartContract).TaxReceived']}]

In [767]:
keys = getAllKeys(peer)[0]
keys

['asset14',
 'asset21',
 'roles_(x509::CN=Alexandre.Payet@babby-0.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-0.kubernetes.research.dev.seeburger.de,O=babby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DEn,)',
 'roles_(x509::CN=Auguste.Boucher@babby-3.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-3.kubernetes.research.dev.seeburger.de,O=babby-3.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DEn,)',
 'roles_(x509::CN=Emma.Steiner@babby-1.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-1.kubernetes.research.dev.seeburger.de,O=babby-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DEn,)',
 'roles_(x509::CN=Pedro.Sanchez@babby-2.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-2.kubernetes.research.dev.seeburger.de,O=babby-2.kubernetes.research.dev.see

In [623]:
getAllRoles(peer)[0][:-1]
#getRoles(peer,name=name)

''

In [448]:
def getPeerUsers(peer):
    strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + peer.name + '.kubernetes.research.dev.seeburger.de/users/']
    print(toCmd(strlist))
    users=[]
    lines=executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 
    for line in lines:
        users.append(line.split('@')[0])
    return users

In [697]:
getPeerUsers(peers[peernames[1]])


/bin/ls /mnt/conf/organizations/peerOrganizations/babby-1.kubernetes.research.dev.seeburger.de/users/
/home/jovyan/work/usr/bin/kubectl exec -t babby-1-56f9fc8dcd-r7ggz -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/babby-1.kubernetes.research.dev.seeburger.de/users/


['Admin',
 'Emma.Steiner',
 'User1',
 'User2',
 'User3',
 'ben',
 'harry',
 'sepp',
 'tom']

In [690]:
strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + peer.name + '.kubernetes.research.dev.seeburger.de/users/']
print(toCmd(strlist))
users=[]
lines=executePeerCmd(peers[peernames[0]].pod_name,toCmd(strlist)).splitlines() 

/bin/ls /mnt/conf/organizations/peerOrganizations/babby-1.kubernetes.research.dev.seeburger.de/users/
/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/babby-1.kubernetes.research.dev.seeburger.de/users/


ls: /mnt/conf/organizations/peerOrganizations/babby-1.kubernetes.research.dev.seeburger.de/users/: No such file or directory
command terminated with exit code 1


In [692]:
peers[peernames[0]].pod_name

'babby-0-7d7bcd4f45-5p57x'

In [580]:
#getSubmittingClientIdentity(peers[peernames[1]],user='ben')[0]

get_certificate_string(peernames[1],user='ben')

'x509::CN=ben@babby-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.babby-1.kubernetes.research.dev.seeburger.de,O=babby-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

In [685]:
name = getSubmittingClientIdentity(peer)[0][:-1].split(',')[0]
group=getSubmittingClientIdentity(peer)[0][:-1].split(',')[1]
msp=getMSPID(peers[peernames[1]])[0][:-1]

name,group,msp

('x509::CN=User1@babby-1.kubernetes.research.dev.seeburger.de',
 'OU=client',
 'babby-1MSP')

### create invoice

In [405]:
get_certificate_string(peernames[1])

'x509::CN=User1@aabby-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.aabby-1.kubernetes.research.dev.seeburger.de,O=aabby-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

In [770]:
peer = peers[peernames[0]]
asset = getEmptyInvoice(peer)[0]
asset['ID']='asset21'
asset['owner'] = getSubmittingClientIdentity(peers[peernames[0]], user='Tony.Dubois')[0]
#asset['buyer'] = getAllRoles(peer)[0][2]['name']
#asset['buyer'] = get_certificate_string(peernames[1],user='ben')
asset['buyer'] = getSubmittingClientIdentity(peers[peernames[1]],user='ben')[0][:-1]
#asset['buyer'] = get_certificate_string(peernames[0])
#createInvoice(peer,getArgList(asset))[1]
#createInvoice(peer,getArgList(asset))

In [861]:
deleteAsset(peer,asset='asset16')[1]

Output()

In [762]:
getSubmittingClientIdentity(peers[peernames[0]], user='User1')[0]

'x509::CN=User1@babby-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.babby-0.kubernetes.research.dev.seeburger.de,O=babby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE\n'

In [778]:
import unicodedata
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [849]:
keys = getAllKeys(peer)[0]
keys

['asset14',
 'asset17',
 'asset3',
 'asset6',
 'roles_(x509::CN=Alexandre.Payet@babby-0.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-0.kubernetes.research.dev.seeburger.de,O=babby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DEn,)',
 'roles_(x509::CN=Auguste.Boucher@babby-3.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-3.kubernetes.research.dev.seeburger.de,O=babby-3.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DEn,)',
 'roles_(x509::CN=Emma.Steiner@babby-1.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-1.kubernetes.research.dev.seeburger.de,O=babby-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DEn,)',
 'roles_(x509::CN=Pedro.Sanchez@babby-2.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-2.kubernetes.research.dev.seeburger.de,O=babby-2.kuber

In [915]:
keys = getAllKeys(peer)[0]
for key in keys:
    if key.startswith('asset'):
        deleteAsset(peer,asset=key)[1]

In [918]:
i=16
invoice = invoices.iloc[16]
seller = strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
buyer = strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
print(i,seller,buyer,'asset'+str(i))
update_invoice(id=i,seller=seller,buyer=buyer)
#createInvoice(get_peer_of_user(seller),getArgList(asset),user=seller)[1]
asset

16 Tony.Dubois Pedro.Sanchez asset16
/bin/ls /mnt/conf/organizations/peerOrganizations/babby-0.kubernetes.research.dev.seeburger.de/users/
/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/babby-0.kubernetes.research.dev.seeburger.de/users/
/bin/ls /mnt/conf/organizations/peerOrganizations/babby-0.kubernetes.research.dev.seeburger.de/users/
/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/babby-0.kubernetes.research.dev.seeburger.de/users/
/bin/ls /mnt/conf/organizations/peerOrganizations/babby-1.kubernetes.research.dev.seeburger.de/users/
/home/jovyan/work/usr/bin/kubectl exec -t babby-1-56f9fc8dcd-r7ggz -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/babby-1.kubernetes.research.dev.seeburger.de/users/
/bin/ls /mnt/conf/organizations/peerOrganizations/babb

{'ID': 'asset16',
 'owner': 'x509::CN=Tony.Dubois@babby-0.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-0.kubernetes.research.dev.seeburger.de,O=babby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
 'buyer': 'x509::CN=Pedro.Sanchez@babby-2.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-2.kubernetes.research.dev.seeburger.de,O=babby-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
 'hash': -1288978081679803368,
 'invoiceNumber': 'FA-2017-0008',
 'tax': '0.00',
 'netto': '2076.76',
 'countryOrigin': 'FR',
 'countryBuyer': 'ES',
 'status': '',
 'received': True,
 'receivedOrder': True,
 'sold': False,
 'claimPaid': False,
 'claimPaidBy': '',
 'taxExemptionReason': 'French VAT exemption according to articles 262 ter I (for products) and/or 283-2 (for services) of "CGI"',
 'taxReceived': False}

In [919]:
asset['taxExemptionReason']='French VAT exemption according to articles 262 ter I (for products) and/or 283-2 (for services) of CGI'
createInvoice(get_peer_of_user(seller),getArgList(asset),user=seller)[1]

/bin/ls /mnt/conf/organizations/peerOrganizations/babby-0.kubernetes.research.dev.seeburger.de/users/
/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/babby-0.kubernetes.research.dev.seeburger.de/users/


Output()

In [916]:
for i in range(0,20):
    try:
        invoice = invoices.iloc[i]
        seller = strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
        buyer = strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
        #print(i,seller,buyer,'asset'+str(i))
        update_invoice(id=i,seller=seller,buyer=buyer)
        print(asset)
        createInvoice(get_peer_of_user(seller),getArgList(asset),user=seller)[0]
    except Exception as e:
        pass

/bin/ls /mnt/conf/organizations/peerOrganizations/babby-0.kubernetes.research.dev.seeburger.de/users/
/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/babby-0.kubernetes.research.dev.seeburger.de/users/
/bin/ls /mnt/conf/organizations/peerOrganizations/babby-0.kubernetes.research.dev.seeburger.de/users/
/home/jovyan/work/usr/bin/kubectl exec -t babby-0-7d7bcd4f45-5p57x -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/babby-0.kubernetes.research.dev.seeburger.de/users/
{'ID': 'asset3', 'owner': 'x509::CN=Tony.Dubois@babby-0.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-0.kubernetes.research.dev.seeburger.de,O=babby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE', 'buyer': 'x509::CN=Alexandre.Payet@babby-0.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de:

In [822]:
def get_peer_of_user(user):
    for key,peer in peers.items():
        if user in getPeerUsers(peer):
            return peer

In [813]:
peers

{'babby-0': <__main__.kubernetespeer at 0x7fe3ccafa910>,
 'babby-1': <__main__.kubernetespeer at 0x7fe3ccafa7c0>,
 'babby-2': <__main__.kubernetespeer at 0x7fe3cd78e9d0>,
 'babby-3': <__main__.kubernetespeer at 0x7fe3cc9bd9d0>}

In [846]:
def update_invoice(id=None,seller='Tony.Dubois',buyer='Stephanie.Hoarau'):
    assign_invoice_to_asset(invoices.iloc[id],asset)
    asset['ID']='asset' + str(id)
    asset['owner'] = getSubmittingClientIdentity(get_peer_of_user(seller), user=seller)[0][:-1]
    asset['buyer'] = getSubmittingClientIdentity(get_peer_of_user(buyer), user=buyer)[0][:-1]
    asset['taxExemptionReason'] = strip_accents(asset['taxExemptionReason'])

In [782]:

    
createInvoice(peer,getArgList(asset),user='Tony.Dubois')[1]

Output()

In [ ]:
getPathValue(invoices.iloc[14],'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/definedTradeContact/')[0]['personName']['value']
getPathValue(invoices.iloc[14],'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/definedTradeContact/')[0]['personName']['value']

### list

In [920]:
#peer=peers[peernames[1]]
#getSubmittingClientIdentity(peer)[0]
listInvoices(peer=peers[peernames[0]], user='Tony.Dubois')[0]
#listInvoices(peer=peers[peernames[0]])[0]
#listInvoices(peer)[0]
#keys

[{'Key': 'asset14',
  'Record': {'ID': 'asset14',
   'owner': 'x509::CN=Tony.Dubois@babby-0.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-0.kubernetes.research.dev.seeburger.de,O=babby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
   'buyer': 'x509::CN=Stephanie.Hoarau@babby-0.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-0.kubernetes.research.dev.seeburger.de,O=babby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
   'hash': -3344848581644188280,
   'invoiceNumber': 'FA-2017-0009',
   'tax': 0,
   'netto': 530.75,
   'countryOrigin': 'FR',
   'countryBuyer': 'RE',
   'status': '',
   'received': False,
   'receivedOrder': False,
   'sold': False,
   'claimPaid': False,
   'claimPaidBy': '',
   'taxExemptionReason': 'Exoneration de TVA selon article 262 I du Code general des impots',
   'taxReceived': False}},
 {'Key': 'asset16',
  'Record': {'ID':

In [864]:
#listInvoice(peer=peers[peernames[1]],  user='ben',id=keys[0])[0]

#assetExists(peer=peers[peernames[1]],id=keys[0])

#listInvoice(peer=peers[peernames[0]],  id=keys[0])[0]
readAssetTest(peer=peers[peernames[0]], asset='asset16')

({'ID': 'asset16',
  'owner': 'x509::CN=Tony.Dubois@babby-0.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-0.kubernetes.research.dev.seeburger.de,O=babby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
  'buyer': 'x509::CN=Pedro.Sanchez@babby-2.kubernetes.research.dev.seeburger.de,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.babby-2.kubernetes.research.dev.seeburger.de,O=babby-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
  'hash': -1288978081679803368,
  'invoiceNumber': 'FA-2017-0008',
  'tax': 0,
  'netto': 2076.76,
  'countryOrigin': 'FR',
  'countryBuyer': 'ES',
  'status': '',
  'received': False,
  'receivedOrder': False,
  'sold': False,
  'claimPaid': False,
  'claimPaidBy': '',
  'taxExemptionReason': 'French VAT exemption according to articles 262 ter I (for products) and/or 283-2 (for services) of CGI',
  'taxReceived': False},
 Output())

In [ ]:
asset=readAsset(peer, asset='asset1')
asset['sold'] = True
updateAsset(peer, asset)

### received

In [664]:
receivedInvoice(peer=peers[peernames[1]],  user='ben',id=keys[0])[1]

Output()

In [545]:
peers[peernames[1]].__dict__

{'name': 'babby-1',
 'pod_name': 'babby-1-56f9fc8dcd-r7ggz',
 'listen_port': '31216',
 'gossip_port': ''}

### load data

In [51]:
import json

def load(filename):
    with open(filename) as json_file:
        return json.load(json_file)

invoices = load('invoices.json')
roles = load('roles.json')
roleTransactions = load('roleTransactions.json')

In [5]:
def get_cert_cn(cert):
    return cert.split('=')[1].split('@')[0]

get_cert_cn(roles[0]['name']), roles[0]['roles']

('Alexandre.Payet', ['Buyer'])

In [6]:
def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(get_cert_cn(person['name']))
    return result        
        
get_person_names_with_role('TaxInspector',roles)        

['Auguste.Boucher']

In [7]:
roles

[{'name': 'x509::CN=Alexandre.Payet@majolieboutique.net,OU=admin,O=majolieboutique.net::CN=ca.majolieboutique.net,O=majolieboutique.net,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR',
  'roles': ['Buyer']},
 {'name': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=lyon-impots.gouv.fr::CN=ca.impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR',
  'roles': ['TaxInspector']},
 {'name': 'x509::CN=Emma.Steiner@austriafactor.at,OU=admin,O=austriafactor.at::CN=ca.austriafactor.at,O=austriafactor.at,L=Graz,ST=Steiermark,C=AT',
  'roles': ['Factor']},
 {'name': 'x509::CN=Pedro.Sanchez@megustaolive.es,OU=admin,O=megustaolive.es::CN=ca.megustaolive.es,O=megustaolive.es,L=Dos Hermanas,ST=Andalusia,C=ES',
  'roles': ['Buyer', 'Seller']},
 {'name': 'x509::CN=Stephanie.Hoarau@hotelsaintdenis.re,OU=admin,O=hotelsaintdenis.re::CN=ca.hotelsaintdenis.re,O=hotelsaintdenis.re,L=Saint Denis,ST=Reunion,C=FR',
  'roles': ['Buyer']},
 {'name': "x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O

In [6]:
if 'TaxInspector' in roles[0]['roles']:
    print()

In [8]:
def invoke(peer,callopt,user='User1'):
    print(user,callopt)
    
def query(peer,callopt,user='User1'):
    print(user,callopt)    
    
class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name    
        
def addPeer(peername):  
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer        
  
def get_peer_of_user(name):
    return kubernetespeer(name=name)    

def addUser(peer,user):
    if peer.name in peerusers:
        peerusers[peer.name].append(user)
    else:
        peerusers[peer.name] = [user]

peers={}
peerusers ={}

addPeer('aubonmoulin')
addPeer('lyon-impots')
addPeer('austriafactor')
addPeer('megustaolive') 
addUser(peers['aubonmoulin'],'Tony.Dubois')    
addUser(peers['aubonmoulin'],'Stephanie.Hoarau')
addUser(peers['aubonmoulin'],'Alexandre.Payet')
addUser(peers['austriafactor'],'Emma.Steiner')    
addUser(peers['megustaolive'],'Pedro.Sanchez')  
addUser(peers['lyon-impots'],'Auguste.Boucher')  

peernames = list(peers.keys())

In [9]:
peerusers
invoices[2]

{'Key': 'asset17',
 'Record': {'ID': 'asset17',
  'owner': "x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=aubonmoulin.fr::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote d'Azur,C=FR",
  'buyer': 'x509::CN=Alexandre.Payet@majolieboutique.net,OU=admin,O=majolieboutique.net::CN=ca.majolieboutique.net,O=majolieboutique.net,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR',
  'hash': -2923879829837918372,
  'invoiceNumber': 'AV-2017-0005',
  'tax': -14.99,
  'netto': -218.48,
  'countryOrigin': 'FR',
  'countryBuyer': 'FR',
  'status': '',
  'received': False,
  'receivedOrder': False,
  'sold': False,
  'claimPaid': False,
  'claimPaidBy': '',
  'taxExemptionReason': '',
  'taxReceived': False}}

## display

In [65]:
#asset=listInvoices(peer=peers[peernames[2]], user='Tony.Dubois')[0][1]['Record']
asset=invoices[3]
_widget = facturx_widgets(asset,'/', hideUnused=True) 
display(_widget)

HTML(value='', description=' ', layout=Layout(width='500px'), style=DescriptionStyle(description_width='250px'…

Text(value='asset3', description='Key', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

HTML(value='Record', description=' ', layout=Layout(width='500px'), style=DescriptionStyle(description_width='…

Text(value='asset3', description='ID', layout=Layout(width='500px'), style=DescriptionStyle(description_width=…

Text(value="x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=aubonmoulin.fr::CN=ca.aubonmoulin.fr,O=aubonmoulin.…

...

Checkbox(value=False, description='sold', layout=Layout(width='500px'), style=DescriptionStyle(description_wid…

Checkbox(value=False, description='claimPaid', layout=Layout(width='500px'), style=DescriptionStyle(descriptio…

Text(value='', description='claimPaidBy', layout=Layout(width='500px'), style=DescriptionStyle(description_wid…

Text(value='', description='taxExemptionReason', layout=Layout(width='500px'), style=DescriptionStyle(descript…

Checkbox(value=False, description='taxReceived', layout=Layout(width='500px'), style=DescriptionStyle(descript…

In [879]:
#listInvoices(peer=peers[peernames[0]], user='Tony.Dubois')[0][0]['Record']

In [57]:
#_widget.container.fields['ID'].widget.disabled=True



In [20]:
owner=_widget.container.fields['owner'].widget.value.split(',')[0].split('=')[1].split('@')[0]
buyer=_widget.container.fields['buyer'].widget.value.split(',')[0].split('=')[1].split('@')[0]
taxinspector=get_person_names_with_role('TaxInspector',roles)
id=_widget.container.fields['ID'].widget.value
peer_buyer=get_peer_of_user(buyer)
peer_owner=get_peer_of_user(owner)
peer_taxinspector=get_peer_of_user(taxinspector)

In [56]:
def disable_all_fields():
    for key,value in _widget.container.fields.items():
        value.widget.disabled=True
        
def enable_fields_for_role(rolename):
    if rolename == 'Buyer':
        _widget.container.fields['received'].widget.disabled=False
        _widget.container.fields['receivedOrder'].widget.disabled=False

    elif  rolename == 'Seller':   
        _widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        _widget.container.fields['owner'].widget.disabled=False
        
    elif  rolename == 'Factor':   
        #_widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        #_widget.container.fields['owner'].widget.disabled=False
                
    elif  rolename == 'TaxInspector':    
        _widget.container.fields['taxReceived'].widget.disabled=False       

In [53]:
disable_all_fields()
enable_fields_for_role('Buyer')

In [54]:
disable_all_fields()
enable_fields_for_role('Seller')

In [55]:
# sold to factor
_widget.container.fields['owner'].widget.value = roles[2]['name']
_widget.container.fields['sold'].widget.value = True

In [57]:
disable_all_fields()
enable_fields_for_role('Factor')

In [61]:
_widget.container.fields['claimPaidBy'].widget.value = 'Christophe Monet'
_widget.container.fields['claimPaid'].widget.value = True

In [66]:
disable_all_fields()
enable_fields_for_role('TaxInspector')

### role buyer

In [49]:


peer=get_peer_of_user(buyer)

def received_changed(b):
    if b['name'] == 'value':
        receivedInvoice(peer_buyer, id=id, user=buyer)[1]
    
def receivedOrder_changed(b):
    if b['name'] == 'value':
        print('changed')
        receivedOrder(peer_buyer, id=id, user=buyer)[1]    
    
_widget.container.fields['received'].widget.observe(received_changed)
_widget.container.fields['receivedOrder'].widget.observe(receivedOrder_changed)

### role owner / seller

In [24]:

def sold_changed(b):
    if b['name'] == 'value':
        print('changed')
        #so(peer, id=id, user=seller)[1]
    
def claimPaid_changed(b):
    if b['name'] == 'value':
        print('changed')
        #receivedOrder(peer, id=id, user=seller)[1]    
    
_widget.container.fields['sold'].widget.observe(received_changed)
_widget.container.fields['claimPaid'].widget.observe(receivedOrder_changed)    

### role tax inspector

In [25]:


def taxReceived_changed(b):
    if b['name'] == 'value':
        taxReceived(peer_taxinspector, id=id, user=taxinspector)[1]
    
_widget.container.fields['taxReceived'].widget.observe(taxReceived_changed)

In [784]:
receivedInvoice(peers[peernames[0]], id='asset14', user='Stephanie.Hoarau')[1]

Output()

### save assets as file

In [416]:
#asset = readAsset(peers[peernames[0]], asset='asset6')

from facturxWidget import facturx_widgets 

import json
with open('asset.json', 'w', encoding='utf-8') as f:
    json.dump(asset, f, ensure_ascii=False, indent=4)

In [ ]:
import json

with open('asset.json') as json_file:
    data = json.load(json_file)

data    

In [921]:
assets=listInvoices(peer=peers[peernames[0]], user='Tony.Dubois')[0]

In [924]:
roleTransactions = getRoleTransactions(peer)[0]
roleTransactions

[{'role': 'Buyer',
  'transactions': ['main.(*SmartContract).ListInvoice',
   'main.(*SmartContract).ListInvoices',
   'main.(*SmartContract).ReceivedInvoice',
   'main.(*SmartContract).ReceivedOrder']},
 {'role': 'Seller',
  'transactions': ['main.(*SmartContract).ListInvoice',
   'main.(*SmartContract).ListInvoices',
   'main.(*SmartContract).CreateInvoice',
   'main.(*SmartContract).TransferInvoice',
   'main.(*SmartContract).ReceivedPayment']},
 {'role': 'Factor',
  'transactions': ['main.(*SmartContract).ListInvoice',
   'main.(*SmartContract).ListInvoices',
   'main.(*SmartContract).TransferInvoice',
   'main.(*SmartContract).ReceivedPayment']},
 {'role': 'TaxInspector',
  'transactions': ['main.(*SmartContract).ListInvoice',
   'main.(*SmartContract).ListInvoices',
   'main.(*SmartContract).TaxReceived']}]

In [929]:
roles=getAllRoles(peer)[0]

In [930]:
import json
with open('roles.json', 'w', encoding='utf-8') as f:
    json.dump(roles, f, ensure_ascii=False, indent=4)

### work

In [233]:
#peers
peer = peers['aabby-0']
#getHostname(peer)

In [263]:
#owner = readAssetTest(peer,'asset6') ['owner']
name = getSubmittingClientIdentity(peer)[:-1]
#appendRole(peer,name=name,role='Seller')
appendRole(peer,name=name,role='TaxInspector')

["TaxInspector","('x509::CN=User1@aabby-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.aabby-0.kubernetes.research.dev.seeburger.de,O=aabby-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE\n',)"]


('', Output())

In [415]:
asset=listInvoices(peer)[0][0]['Record']

In [602]:
#deleteAsset(peer,asset="roles")[1]
#assetExists(peer,asset="roles")
#readAsset(peer,asset="roles")[1]
#getAllRoles(peer)
#getRoles(peer,name=name)
#getSubmittingClientIdentity(peer)

In [600]:
#peer.pod_name
deleteAsset(peer, asset=keys[3])

('', Output())

## invoice usecase (Simple)

In [ ]:
def getPeer0Address(peer):
    addr='peer0.kubernetes.research.dev.seeburger.de'
    return str(addr + ':' + peer.listen_port)

def readAsset(peer, asset='asset1'):
    #callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    callopt= '{"function":"ReadAsset","Args":[' + '\"' + asset + '\"' + ']}'
    cmd0 = str('\'' + callopt + "\'")
    try:
        #cmd1= 'CORE_PEER_ADDRESS=\"peer0.kubernetes.research.dev.seeburger.de:31866\" && ' 
        #cmd1 = 'CORE_PEER_ADDRESS=\"' + getPeer0Address(peer) + '\" && '
        cmd1 = 'CORE_PEER_MSPCONFIGPATH=' + path + '\" && '
        #cmd2 = ['peer', 'chaincode','query', '-C', channel_name, '-n', 'basic', '-c']
        cmd2 = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem', '-C',channel_name, '-n', 'basic', '-c']
   
        cmd = toCmd(cmd2) + ' ' + cmd0
        print(cmd)
        #output = executePeerCmd(peer.pod_name, cmd1 + cmd)
        #print(output)
        if output[0] == '{':
            return json.loads(output)
        return cmd
    except Exception as e:
        print(e)
        
asset = readAsset(peers[pernames[0]], 'asset1')
executePeerCmd(peers[pernames[0]].pod_name,asset)

In [ ]:

pernames = list(peers.keys())
chaincode_add_invoice(peers[pernames[0]],channel_name,'046',pernames[1])

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -t galya-0-6cc848b49-pv2dn -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh super13 006 x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US

In [ ]:
get_certificate_string(pernames[1])

#'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

#, 'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

In [ ]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  $CHANNEL_NAME $INVOICE_ID
def chaincode_read_invoice(peer,channel_name):
    strlist = ['/mnt/conf/peer/get-my-invoices.sh', channel_name]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 
chaincode_read_invoice(peers[pernames[0]],channel_name)

In [ ]:
def chaincode_transfer_invoice(peer,channel_name,invoice_id,new_owner):
    strlist = ['/mnt/conf/peer/transfer_invoice.sh', channel_name,invoice_id,get_certificate_string(new_owner)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

pernames = list(peers.keys())
chaincode_transfer_invoice(peers[pernames[0]],channel_name,'045',pernames[2])

In [ ]:
get_certificate_string(pernames[2])
NEW_OWNER='x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/transfer_invoice.sh  super43 045 $NEW_OWNER

In [ ]:
kubectl exec --stdin --tty 'lalya-0-7c887cffb8-fpw7h' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-1-7c76dff8c-pzx78' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-2-5f8b45dbc4-52tt7' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045


In [ ]:
peers[pernames[2]].pod_name

## add channel

In [23]:
orderer.pod_name

'orderer-org1-scray-org-5cb87587c4-q95mm'

In [25]:
!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-5cb87587c4-q95mm -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super55 orderer.example.com 30081

2021-11-11 08:32:57.480 UTC [common.tools.configtxgen] main -> INFO 001 Loading configuration
2021-11-11 08:32:57.500 UTC [common.tools.configtxgen.localconfig] Load -> INFO 002 Loaded configuration: /mnt/conf/orderer/configtx.yaml
2021-11-11 08:32:57.500 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 003 Generating new channel configtx
2021-11-11 08:32:57.503 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 004 Writing new channel tx
2021-11-11 08:32:57.568 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-11-11 08:32:57.618 UTC [cli.common] readBlock -> INFO 002 Expect block, but got status: &{NOT_FOUND}
2021-11-11 08:32:57.621 UTC [channelCmd] InitCmdFactory -> INFO 003 Endorser and orderer connections initialized
2021-11-11 08:32:57.823 UTC [cli.common] readBlock -> INFO 004 Expect block, but got status: &{SERVICE_UNAVAILABLE}
2021-11-11 08:32:57.829 UTC [channelCmd] InitCmdFactory -> INFO 005 Endorser and or

In [ ]:
add_channel('super20')

In [ ]:
name = 'super30'
cmd = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050'])
cmd, orderer.pod_name

In [ ]:
executePeerCmd(orderer.pod_name, cmd, cli='scray-orderer-cli')

In [ ]:
executePeerCmd(orderer.pod_name, 'peer channel list', cli='scray-orderer-cli')

In [ ]:
cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh super20 orderer.example.com 7050']
executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

In [ ]:
#cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c','/mnt/conf/orderer/scripts/create_channel.sh super7 orderer.example.com 7050']

#cmd2 = toCmd(['ls','-a'])
cmd2 = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', 'super7', 'orderer.example.com', '7050'])

cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c',cmd2]


print(toCmd(cmd1))
#subprocess.check_output(cmd1)
subprocess.run(cmd1, stdout = subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -it orderer-org1-scray-org-5f97c57d44-l6d5l -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super6 orderer.example.com 7050


In [ ]:
orderer.pod_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty 'orderer-org1-scray-org-5f97c57d44-j5mbs' -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh 'super' orderer.example.com '7050'

In [ ]:
# Create Peer

peername='dalya8'
addPeer(peername)
peers[peername].__dict__

In [4]:
##


In [ ]:
executeKubectlCmd(['get', 'pods','-l', 'app=orderer-org1-scray-org','-o','jsonpath=\'{.items[*].status.podIP}\''])  

In [ ]:
peers[peername].__dict__

In [ ]:
orderer.__dict__

## Export channel configuration

In [ ]:
#https://github.com/scray/scray/blob/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/README.md#export-data

SHARED_FS_HOST='10.14.128.38:30080'
_result = SHARED_FS_HOST + '/channel/configuration/$CHANNEL_NAME/config.json'

#kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/publish_channel_conf.sh $CHANNEL_NAME $SHARED_FS_HOST

def export_channel_configuration(peer,channel_name,SHARED_FS_HOST):
    strlist = ['/mnt/conf/orderer/scripts/publish_channel_conf.sh', channel_name,SHARED_FS_HOST]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

#_peer = peers[list(peers.keys())[0]]
_peer = orderer
channel_name = 'super'
export_channel_configuration(_peer,channel_name,SHARED_FS_HOST)

In [ ]:
#list(peers)[0]


### display certificate 

In [393]:
# Cerificates
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.x509.oid import NameOID

#https://stackoverflow.com/questions/16899247/how-can-i-decode-a-ssl-certificate-using-python

def getCrt(org='',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    cmd = ['curl', '--user',user + ':' + pwd, 
           'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt']
    res = executeLocalCmd(cmd)
    if 'CERTIFICATE' in str(res):
        return res
       
def getOrgCrt(org=''):    
    pem_data=getCrt(org)
    if pem_data:
        return x509.load_pem_x509_certificate(pem_data, default_backend())

    
def getcrt(peer,user='User1'):
    strlist = ['cat',  ' /tmp/crt_target/hans/user.crt']
    
    #           '/mnt/conf/organizations/peerOrganizations/aabby-0.kubernetes.research.dev.seeburger.de/users/hans@aabby-0.kubernetes.research.dev.seeburger.de/msp/signcerts/hans@aabby-0.kubernetes.research.dev.seeburger.de-cert.pem']
    cmd = toCmd(strlist)
    #print(cmd)  
    output = executePeerCmd(peer.pod_name,  cmd)
    return output,_out    

In [403]:
crt=getcrt(peer)[0][:-1]
str(crt)

/home/jovyan/work/usr/bin/kubectl exec -t aabby-0-6859c54c76-gvdqw -c scray-peer-cli -- /bin/sh -c cat  /tmp/crt_target/hans/user.crt


'-----BEGIN CERTIFICATE-----\nMIIB7DCCAZICFAOrSfcVRsaXLX5qOSp/qCnY2XahMAoGCCqGSM49BAMCMIGgMQsw\nCQYDVQQGEwJERTEOMAwGA1UECBMFQmFkZW4xEDAOBgNVBAcTB0JyZXR0ZW4xNTAz\nBgNVBAoTLHphbHlhLTAua3ViZXJuZXRlcy5yZXNlYXJjaC5kZXYuc2VlYnVyZ2Vy\nLmRlMTgwNgYDVQQDEy9jYS56YWx5YS0wLmt1YmVybmV0ZXMucmVzZWFyY2guZGV2\nLnNlZWJ1cmdlci5kZTAeFw0yMTExMTAxNDUxMzBaFw0yMjExMDUxNDUxMzBaMFAx\nDjAMBgNVBAMMBWhhbnMgMS4wLAYDVQQKDCVrdWJlcm5ldGVzLnJlc2VhcmNoLmRl\ndi5zZWVidXJnZXIuZGUgMQ4wDAYDVQQLDAVhZG1pbjBZMBMGByqGSM49AgEGCCqG\nSM49AwEHA0IABOVYd8LrcD2EcRciEFOaftZSQKyThsDf+vQOS5V8nLWA6+QNLTpk\ni4sDHy+EEM1VgbvJLwQU39Rzb264fcNA8mowCgYIKoZIzj0EAwIDSAAwRQIgfpOO\nToJT5RzQJraozc6dY2hZu0+7Oe/prmwAyY6W/tsCIQC8hyF5IibZ3A5hdqki1Qif\n8tnOVBiJFP+yUAYlHCOakw==\n-----END CERTIFICATE-----'

In [ ]:
x509.load_pem_x509_certificate(crt, default_backend())